In [ ]:
import os
import sys
import asyncio
from async_timeout import timeout
import json

# Add the project root directory to the Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.insert(0, project_root)

from src.clients.morpho_client import MorphoClient


async def run_with_timeout(coro, timeout_seconds=10):
    async with timeout(timeout_seconds):
        return await coro


In [ ]:
# Setup client
RPC = "ETHEREUM"  # ARBITRUM or ETHEREUM
RPC_URL = os.getenv(f"{RPC}_RPC_URL")
user_address = os.getenv('WALLET_ADDRESS')

# Initialize client
morpho_client = MorphoClient(wallet_address=user_address,
                             private_key=os.getenv('PRIVATE_WALLET_KEY'),
                             network=RPC,
                             rpc_url=RPC_URL)

In [ ]:
# Test get the smart contracts:
morpho_blue = morpho_client._get_morpho_contract()
print(morpho_blue.address)

morpho_irm = morpho_client._get_irm_contract()
print(morpho_irm.address)

### Fetch a single market information

In [ ]:
market_key = 'wstETH-USDT1' # see utils.morpho_markets for all available markets
market_id = morpho_client.get_market_info(market_key)
print(market_id)

In [ ]:
markets_data = await run_with_timeout(morpho_client.fetch_market_data(market_id, user_address))
markets_data

### Fetch many markets information

In [ ]:
# List of market keys you want to fetch data for
market_keys = ['wstETH-USDT1', 'wstETH-USDT2', 'wstETH-USDC', 'wstETH-DAI', 'wstETH-USDA']

# Get market IDs for all markets
market_ids = [morpho_client.get_market_info(key) for key in market_keys]

# Fetch data for all markets
markets_data = await run_with_timeout(morpho_client.fetch_all_markets_data(market_ids, user_address))

# Print or process the results
for key, data in zip(market_keys, markets_data.values()):
    print(f"Market: {key}")
    print(json.dumps(data, indent=2))
    print("\n")